In [12]:
import pandas as pd
import geopandas as gpd
import scipy.stats as ss
from pyproj import Proj, transform
from shapely.geometry import Point
from lets_plot.plot import *

# Inset map of Kotlin #

[The geodata is provided by © OpenStreetMap contributors and is made available here under the Open Database License (ODbL)](https://www.openstreetmap.org/copyright).

In [13]:
def lonlat_to_xy(lon, lat, proj='epsg:3857'):
    LONLAT_PROJ = 'epsg:4326'
    
    return transform(Proj(init=LONLAT_PROJ), Proj(init=proj), lon, lat)

In [14]:
DISTRICTS = [ 1114193, 1114252, 1114354, 1114806, 1114809, 337424, 1114895, 363103, 1115082, 1115366, 338636, 368287, 1114905, 367375, 1115367, 338635, 369514, 1114902 ]
KOTLIN_ID = 1115082

In [15]:
gdf = gpd.read_file('data/spb.shp')

In [16]:
data = { 'osm_id' : [ ], 'lon' : [ ], 'lat' : [ ] }

for i, row in gdf.iterrows():
    for lon, lat in row['geometry'].exterior.coords:
        osm_id = int(row['OSM_ID'])
        if not osm_id in DISTRICTS:
            continue
        data['osm_id'].append(osm_id)
        data['lon'].append(lon)
        data['lat'].append(lat)

In [17]:
df = pd.DataFrame(data)
df['x'], df['y'] = lonlat_to_xy(df['lon'].tolist(), df['lat'].tolist())
df_kotlin = df[df['osm_id'] == KOTLIN_ID]

In [18]:
p1 = ggplot()
p1 += geom_polygon(aes(x='x', y='y'), data=df, color='black', fill='white')
p1 += geom_rect(xmin=df_kotlin['x'].min(), xmax=df_kotlin['x'].max(), ymin=df_kotlin['y'].min(), ymax=df_kotlin['y'].max(), color='red', size=.75, alpha=0)
p1 += theme(axis_title=element_blank(), axis_text=element_blank(), axis_ticks=element_blank(), axis_line=element_blank(), legend_position='none')

In [19]:
kotlin_polygon = gdf[gdf['OSM_ID'] == KOTLIN_ID]['geometry'].iloc[1]
naval_cathedral_centroid = Point(29.777771, 59.991738)

In [20]:
VIKINGS_AGE = (793, 1067)
COV = [[0.003, -0.002, 0], [-0.002, 0.003, 0], [0, 0, 1000]]
SIZE = 200

viking_camps = ss.multivariate_normal.rvs(mean=[naval_cathedral_centroid.x, naval_cathedral_centroid.y, (VIKINGS_AGE[0] + VIKINGS_AGE[1]) / 2], cov=COV, size=SIZE)
df_random_viking_camps = pd.DataFrame(viking_camps, columns=['lon', 'lat', 'year'])
df_random_viking_camps['year'] = df_random_viking_camps['year'].astype(int)
df_timely_viking_camps = df_random_viking_camps[df_random_viking_camps.apply(lambda vc: vc['year'] in range(VIKINGS_AGE[0], VIKINGS_AGE[1]), axis=1)]
df_viking_camps = df_timely_viking_camps[df_timely_viking_camps.apply(lambda vc: kotlin_polygon.contains(Point(vc['lon'], vc['lat'])), axis=1)]
df_viking_camps = df_viking_camps.assign(**dict(zip(['x', 'y'], lonlat_to_xy(df_viking_camps['lon'].tolist(), df_viking_camps['lat'].tolist()))))

In [21]:
p2 = ggplot()
p2 += geom_polygon(aes(x='x', y='y'), data=df_kotlin, color='#31a354', fill='#e5f5e0')
p2 += geom_point(aes(x='x', y='y', color='year'), data=df_viking_camps)
p2 += scale_color_gradient(low='#662506', high='#fec44f', name='Year')
p2 += ggtitle('Viking Camps on Kotlin from %s to %s A.D.' % VIKINGS_AGE)
p2 += theme(axis_title=element_blank(), axis_text=element_blank(), axis_ticks=element_blank(), axis_line=element_blank())

In [22]:
bunch = GGBunch()
bunch.add_plot(p2, 0, 0, 800, 600)
bunch.add_plot(p1, 600, 0, 200, 150)
bunch.show()